# פרויקט סיום ניתוח נתוני עתק, המכללה האקדמית צפת
# מגישים: אביב יהונתן אליהו, אריאל גולדווסר, מתן אסרף

In [0]:
# import all needed functions
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier,DecisionTreeClassifier,LinearSVC,GBTClassifier,FMClassifier
from pyspark.ml import Pipeline
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,OneHotEncoder,StringIndexer)
from pyspark.ml.evaluation import BinaryClassificationEvaluator,MulticlassClassificationEvaluator
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from pyspark.sql.types import StructField,StringType,IntegerType,StructType
import pyspark.pandas as pd
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt 
import pyspark.pandas as ps
ps.set_option('compute.ops_on_diff_frames', True)

spark = SparkSession.builder.appName("Credit_Approval").getOrCreate()

In [0]:
# read relevant file
credit = spark.read.csv("/FileStore/tables/clean_dataset.csv",inferSchema=True,header=True)

In [0]:
#lets have a look at what we have
credit

Out[3]: DataFrame[Gender: int, Age: double, Debt: double, Married: int, BankCustomer: int, Industry: string, Ethnicity: string, YearsEmployed: double, PriorDefault: int, Employed: int, CreditScore: int, DriversLicense: int, Citizen: string, ZipCode: int, Income: int, Approved: int]

In [0]:
final_data = credit.dropna(how="any")

In [0]:
credit.printSchema()

root
 |-- Gender: integer (nullable = true)
 |-- Age: double (nullable = true)
 |-- Debt: double (nullable = true)
 |-- Married: integer (nullable = true)
 |-- BankCustomer: integer (nullable = true)
 |-- Industry: string (nullable = true)
 |-- Ethnicity: string (nullable = true)
 |-- YearsEmployed: double (nullable = true)
 |-- PriorDefault: integer (nullable = true)
 |-- Employed: integer (nullable = true)
 |-- CreditScore: integer (nullable = true)
 |-- DriversLicense: integer (nullable = true)
 |-- Citizen: string (nullable = true)
 |-- ZipCode: integer (nullable = true)
 |-- Income: integer (nullable = true)
 |-- Approved: integer (nullable = true)



### אם כך עמודות Industry, Ethnicity ו- Citizen
### הן עמודות מסוג מחרוזת אשר צריך לבצע עליהן עיבוד לצורה מספרית לצורך שימוש בעמודות אלו במודל החיזוי

In [0]:
credit.columns

Out[6]: ['Gender',
 'Age',
 'Debt',
 'Married',
 'BankCustomer',
 'Industry',
 'Ethnicity',
 'YearsEmployed',
 'PriorDefault',
 'Employed',
 'CreditScore',
 'DriversLicense',
 'Citizen',
 'ZipCode',
 'Income',
 'Approved']

In [0]:
#for Industry

industry_index = StringIndexer(inputCol='Industry',outputCol='IndustryIndex')

industry_encoder = OneHotEncoder(inputCol='IndustryIndex',outputCol="IndustryVec")

# outcome = IndustryVec

In [0]:
#for Ethnicity

ethnicity_index = StringIndexer(inputCol='Ethnicity',outputCol='EthnicityIndex')

ethnicity_encoder = OneHotEncoder(inputCol='EthnicityIndex',outputCol="EthnicityVec")

# outcome = EthnicityVec

In [0]:
#for Citizen

citizen_index = StringIndexer(inputCol='Citizen',outputCol='CitizenIndex')

citizen_encoder = OneHotEncoder(inputCol='CitizenIndex',outputCol="CitizenVec")

# outcome = CitizenVec

In [0]:
assembler = VectorAssembler(inputCols=['Gender','Age','Debt','Married','BankCustomer','YearsEmployed','PriorDefault',
 'Employed','CreditScore','DriversLicense','ZipCode','Income','IndustryVec',"EthnicityVec","CitizenVec"], outputCol = "features")

In [0]:
#all the models we will use
lr_model = LogisticRegression(featuresCol="features", labelCol="Approved")

rf_model = RandomForestClassifier(featuresCol="features", labelCol="Approved",numTrees=20)

dt_model = DecisionTreeClassifier(featuresCol="features", labelCol="Approved")

gbt_model = GBTClassifier(featuresCol="features", labelCol="Approved")

svm_model = LinearSVC(featuresCol="features", labelCol="Approved")

# new algorithm!

fm_model = FMClassifier(featuresCol="features", labelCol="Approved", stepSize=0.001)

In [0]:
# dictionary for each model
lr={'acc':0,'f1':0,'recall':0,'precision':0,'auc':0,'specifity':0,'fp':0,'tn':0}
rf={'acc':0,'f1':0,'recall':0,'precision':0,'auc':0,'specifity':0,'fp':0,'tn':0}
dt={'acc':0,'f1':0,'recall':0,'precision':0,'auc':0,'specifity':0,'fp':0,'tn':0}
gbt={'acc':0,'f1':0,'recall':0,'precision':0,'auc':0,'specifity':0,'fp':0,'tn':0}
svm={'acc':0,'f1':0,'recall':0,'precision':0,'auc':0,'specifity':0,'fp':0,'tn':0}
fm={'acc':0,'f1':0,'recall':0,'precision':0,'auc':0,'specifity':0,'fp':0,'tn':0}

# pipeline for each model
pipline_lr = Pipeline(stages = [industry_index,citizen_index,ethnicity_index,industry_encoder,citizen_encoder,ethnicity_encoder,assembler,lr_model])
pipline_rf = Pipeline(stages = [industry_index,citizen_index,ethnicity_index,industry_encoder,citizen_encoder,ethnicity_encoder,assembler,rf_model])
pipline_gbt = Pipeline(stages = [industry_index,citizen_index,ethnicity_index,industry_encoder,citizen_encoder,ethnicity_encoder,assembler,gbt_model])
pipline_dt = Pipeline(stages = [industry_index,citizen_index,ethnicity_index,industry_encoder,citizen_encoder,ethnicity_encoder,assembler,dt_model])
pipline_svm = Pipeline(stages = [industry_index,citizen_index,ethnicity_index,industry_encoder,citizen_encoder,ethnicity_encoder,assembler,svm_model])
pipline_fm = Pipeline(stages = [industry_index,citizen_index,ethnicity_index,industry_encoder,citizen_encoder,ethnicity_encoder,assembler,fm_model])

for i in range(10):
    train,test = final_data.randomSplit([0.7,0.3])

    # fit each model
    fit_model_lr = pipline_lr.fit(train)
    fit_model_rf = pipline_rf.fit(train)
    fit_model_gbt = pipline_gbt.fit(train)
    fit_model_dt = pipline_dt.fit(train)
    fit_model_svm = pipline_svm.fit(train)
    fit_model_fm = pipline_fm.fit(train)

    # results of each model
    results_lr = fit_model_lr.transform(test)
    results_lr = (results_lr.withColumnRenamed('Approved','Approved_lr').withColumnRenamed('prediction','prediction_lr'))

    results_rf = fit_model_rf.transform(test)
    results_rf = (results_rf.withColumnRenamed('Approved','Approved_rf').withColumnRenamed('prediction','prediction_rf'))

    results_gbt = fit_model_gbt.transform(test)
    results_gbt = (results_gbt.withColumnRenamed('Approved','Approved_gbt').withColumnRenamed('prediction','prediction_gbt'))

    results_dt = fit_model_dt.transform(test)
    results_dt = (results_dt.withColumnRenamed('Approved','Approved_dt').withColumnRenamed('prediction','prediction_dt'))
    
    results_svm = fit_model_svm.transform(test)
    results_svm = (results_svm.withColumnRenamed('Approved','Approved_svm').withColumnRenamed('prediction','prediction_svm'))

    results_fm = fit_model_fm.transform(test)
    results_fm = (results_fm.withColumnRenamed('Approved','Approved_fm').withColumnRenamed('prediction','prediction_fm'))

    # final results of each model
    final_results_lr = results_lr.select('Approved_lr','prediction_lr')
    final_results_rf = results_rf.select('Approved_rf','prediction_rf')
    final_results_gbt = results_gbt.select('Approved_gbt','prediction_gbt')
    final_results_dt = results_dt.select('Approved_dt','prediction_dt')
    final_results_svm = results_svm.select('Approved_svm','prediction_svm')
    final_results_fm = results_fm.select('Approved_fm','prediction_fm')

    # evaluation of each model
    my_eval_lr = BinaryClassificationEvaluator(rawPredictionCol='prediction_lr',labelCol='Approved_lr')
    my_eval_rf = BinaryClassificationEvaluator(rawPredictionCol='prediction_rf',labelCol='Approved_rf')
    my_eval_gbt = BinaryClassificationEvaluator(rawPredictionCol='prediction_gbt',labelCol='Approved_gbt')
    my_eval_dt = BinaryClassificationEvaluator(rawPredictionCol='prediction_dt',labelCol='Approved_dt')
    my_eval_svm = BinaryClassificationEvaluator(rawPredictionCol='prediction_svm',labelCol='Approved_svm')
    my_eval_fm = BinaryClassificationEvaluator(rawPredictionCol='prediction_fm',labelCol='Approved_fm')

    # confusion matrix of each model
    y_test_lr = test.select(['Approved']).collect()
    y_pred_lr = final_results_lr.select(['prediction_lr']).collect()
    confusion_lr = confusion_matrix(y_test_lr, y_pred_lr)

    y_test_rf = test.select(['Approved']).collect()
    y_pred_rf = final_results_rf.select(['prediction_rf']).collect()
    confusion_rf = confusion_matrix(y_test_rf, y_pred_rf)

    y_test_gbt = test.select(['Approved']).collect()
    y_pred_gbt = final_results_gbt.select(['prediction_gbt']).collect()
    confusion_gbt = confusion_matrix(y_test_gbt, y_pred_gbt)

    y_test_dt = test.select(['Approved']).collect()
    y_pred_dt = final_results_dt.select(['prediction_dt']).collect()
    confusion_dt = confusion_matrix(y_test_dt, y_pred_dt)

    y_test_svm = test.select(['Approved']).collect()
    y_pred_svm = final_results_svm.select(['prediction_svm']).collect()
    confusion_svm = confusion_matrix(y_test_svm, y_pred_svm)

    y_test_fm = test.select(['Approved']).collect()
    y_pred_fm = final_results_fm.select(['prediction_fm']).collect()
    confusion_fm = confusion_matrix(y_test_fm, y_pred_fm)


    # calculate each model's results
    eval_acc_lr = MulticlassClassificationEvaluator(predictionCol='prediction_lr', labelCol='Approved_lr',metricName='accuracy')
    eval_f1_lr = MulticlassClassificationEvaluator(predictionCol='prediction_lr', labelCol='Approved_lr',metricName='f1')
    eval_recall_lr = MulticlassClassificationEvaluator(predictionCol='prediction_lr', labelCol='Approved_lr',metricName='recallByLabel')
    eval_precision_lr = MulticlassClassificationEvaluator(predictionCol='prediction_lr', labelCol='Approved_lr',metricName='precisionByLabel')

    eval_acc_rf = MulticlassClassificationEvaluator(predictionCol='prediction_rf', labelCol='Approved_rf',metricName='accuracy')
    eval_f1_rf = MulticlassClassificationEvaluator(predictionCol='prediction_rf', labelCol='Approved_rf',metricName='f1')
    eval_recall_rf = MulticlassClassificationEvaluator(predictionCol='prediction_rf', labelCol='Approved_rf',metricName='recallByLabel')
    eval_precision_rf = MulticlassClassificationEvaluator(predictionCol='prediction_rf', labelCol='Approved_rf',metricName='precisionByLabel')

    eval_acc_gbt = MulticlassClassificationEvaluator(predictionCol='prediction_gbt', labelCol='Approved_gbt',metricName='accuracy')
    eval_f1_gbt = MulticlassClassificationEvaluator(predictionCol='prediction_gbt', labelCol='Approved_gbt',metricName='f1')
    eval_recall_gbt = MulticlassClassificationEvaluator(predictionCol='prediction_gbt', labelCol='Approved_gbt',metricName='recallByLabel')
    eval_precision_gbt = MulticlassClassificationEvaluator(predictionCol='prediction_gbt', labelCol='Approved_gbt',metricName='precisionByLabel')

    eval_acc_dt = MulticlassClassificationEvaluator(predictionCol='prediction_dt', labelCol='Approved_dt',metricName='accuracy')
    eval_f1_dt = MulticlassClassificationEvaluator(predictionCol='prediction_dt', labelCol='Approved_dt',metricName='f1')
    eval_recall_dt = MulticlassClassificationEvaluator(predictionCol='prediction_dt', labelCol='Approved_dt',metricName='recallByLabel')
    eval_precision_dt = MulticlassClassificationEvaluator(predictionCol='prediction_dt', labelCol='Approved_dt',metricName='precisionByLabel')

    eval_acc_svm = MulticlassClassificationEvaluator(predictionCol='prediction_svm', labelCol='Approved_svm',metricName='accuracy')
    eval_f1_svm = MulticlassClassificationEvaluator(predictionCol='prediction_svm', labelCol='Approved_svm',metricName='f1')
    eval_recall_svm = MulticlassClassificationEvaluator(predictionCol='prediction_svm', labelCol='Approved_svm',metricName='recallByLabel')
    eval_precision_svm = MulticlassClassificationEvaluator(predictionCol='prediction_svm', labelCol='Approved_svm',metricName='precisionByLabel')

    eval_acc_fm = MulticlassClassificationEvaluator(predictionCol='prediction_fm', labelCol='Approved_fm',metricName='accuracy')
    eval_f1_fm = MulticlassClassificationEvaluator(predictionCol='prediction_fm', labelCol='Approved_fm',metricName='f1')
    eval_recall_fm = MulticlassClassificationEvaluator(predictionCol='prediction_fm', labelCol='Approved_fm',metricName='recallByLabel')
    eval_precision_fm = MulticlassClassificationEvaluator(predictionCol='prediction_fm', labelCol='Approved_fm',metricName='precisionByLabel')

    # save results of each model
    accuracy_lr = eval_acc_lr.evaluate(final_results_lr)
    f1_lr = eval_f1_lr.evaluate(final_results_lr)
    precision_lr = eval_precision_lr.evaluate(final_results_lr)
    recall_lr = eval_recall_lr.evaluate(final_results_lr)
    auc_lr = my_eval_lr.evaluate(final_results_lr)

    accuracy_rf = eval_acc_rf.evaluate(final_results_rf)
    f1_rf = eval_f1_rf.evaluate(final_results_rf)
    precision_rf = eval_precision_rf.evaluate(final_results_rf)
    recall_rf = eval_recall_rf.evaluate(final_results_rf)
    auc_rf = my_eval_rf.evaluate(final_results_rf)

    accuracy_gbt = eval_acc_gbt.evaluate(final_results_gbt)
    f1_gbt = eval_f1_gbt.evaluate(final_results_gbt)
    precision_gbt = eval_precision_gbt.evaluate(final_results_gbt)
    recall_gbt = eval_recall_gbt.evaluate(final_results_gbt)
    auc_gbt = my_eval_gbt.evaluate(final_results_gbt)

    accuracy_dt = eval_acc_dt.evaluate(final_results_dt)
    f1_dt = eval_f1_dt.evaluate(final_results_dt)
    precision_dt = eval_precision_dt.evaluate(final_results_dt)
    recall_dt = eval_recall_dt.evaluate(final_results_dt)
    auc_dt = my_eval_dt.evaluate(final_results_dt)

    accuracy_svm = eval_acc_svm.evaluate(final_results_svm)
    f1_svm = eval_f1_svm.evaluate(final_results_svm)
    precision_svm = eval_precision_svm.evaluate(final_results_svm)
    recall_svm = eval_recall_svm.evaluate(final_results_svm)
    auc_svm = my_eval_svm.evaluate(final_results_svm)

    accuracy_fm = eval_acc_fm.evaluate(final_results_fm)
    f1_fm = eval_f1_fm.evaluate(final_results_fm)
    precision_fm = eval_precision_fm.evaluate(final_results_fm)
    recall_fm = eval_recall_fm.evaluate(final_results_fm)
    auc_fm = my_eval_fm.evaluate(final_results_fm)

    #add values to each model's dictionary
    lr['acc'] = lr['acc'] +accuracy_lr
    lr['f1'] = lr['f1'] +f1_lr
    lr['recall'] = lr['recall'] +recall_lr
    lr['precision'] = lr['precision'] +precision_lr
    lr['auc'] = lr['auc'] +auc_lr
    lr['fp'] = lr['fp'] + confusion_lr[0][1]
    lr['tn'] = lr['tn'] + confusion_lr[1][1]
    lr["specifity"] = lr['tn']/(lr['tn']+lr['fp'])

    rf['acc'] = rf['acc'] +accuracy_rf
    rf['f1'] = rf['f1'] +f1_rf
    rf['recall'] = rf['recall'] +recall_rf
    rf['precision'] = rf['precision'] +precision_rf
    rf['auc'] = rf['auc'] +auc_rf
    rf['fp'] = rf['fp'] + confusion_rf[0][1]
    rf['tn'] = rf['tn'] + confusion_rf[1][1]
    rf["specifity"] = rf['tn']/(rf['tn']+rf['fp'])

    gbt['acc'] = gbt['acc'] +accuracy_gbt
    gbt['f1'] = gbt['f1'] +f1_gbt
    gbt['recall'] = gbt['recall'] +recall_gbt
    gbt['precision'] = gbt['precision'] +precision_gbt
    gbt['auc'] = gbt['auc'] +auc_gbt
    gbt['fp'] = gbt['fp'] + confusion_gbt[0][1]
    gbt['tn'] = gbt['tn'] + confusion_gbt[1][1]
    gbt["specifity"] = gbt['tn']/(gbt['tn']+gbt['fp'])

    dt['acc'] = dt['acc'] +accuracy_dt
    dt['f1'] = dt['f1'] +f1_dt
    dt['recall'] = dt['recall'] +recall_dt
    dt['precision'] = dt['precision'] +precision_dt
    dt['auc'] = dt['auc'] +auc_dt
    dt['fp'] = dt['fp'] + confusion_dt[0][1]
    dt['tn'] = dt['tn'] + confusion_dt[1][1]
    dt["specifity"] = dt['tn']/(dt['tn']+dt['fp'])

    svm['acc'] = svm['acc'] +accuracy_svm
    svm['f1'] = svm['f1'] +f1_svm
    svm['recall'] = svm['recall'] +recall_svm
    svm['precision'] = svm['precision'] +precision_svm
    svm['auc'] = svm['auc'] +auc_svm
    svm['fp'] = svm['fp'] + confusion_svm[0][1]
    svm['tn'] = svm['tn'] + confusion_svm[1][1]
    svm["specifity"] = svm['tn']/(svm['tn']+svm['fp'])

    fm['acc'] = fm['acc'] +accuracy_fm
    fm['f1'] = fm['f1'] +f1_fm
    fm['recall'] = fm['recall'] +recall_fm
    fm['precision'] = fm['precision'] +precision_fm
    fm['auc'] = fm['auc'] +auc_fm
    fm['fp'] = fm['fp'] + confusion_fm[0][1]
    fm['tn'] = fm['tn'] + confusion_fm[1][1]
    fm["specifity"] = fm['tn']/(fm['tn']+fm['fp'])

In [0]:
# create dataframe for each model

df_lr = pd.DataFrame(lr,index=["Logistic Regression"])
df_rf = pd.DataFrame(rf,index=["Random Forest"])
df_dt = pd.DataFrame(dt,index=["Decision Tree"])
df_gbt = pd.DataFrame(gbt,index=["Gradient Boosting Tree"])
df_svm = pd.DataFrame(svm,index=["Support Vector Machine"])
df_fm = pd.DataFrame(fm,index=["Factorization machines"])

In [0]:
# create a combined DF of all the models
models_df = pd.concat([df_lr,df_rf,df_dt,df_gbt,df_svm,df_fm])
models_df = models_df/10
models_df['specifity'] = models_df['specifity']*10

#rename the index
models_df = models_df.rename_axis('Model Name')

#show model
models_df

,acc,f1,recall,precision,auc,specifity,fp,tn
Model Name,,,,,,,,
Logistic Regression,0.852998,0.853307,0.847327,0.888667,0.853807,0.812308,18.3,79.2
Random Forest,0.869599,0.869723,0.874249,0.894116,0.868986,0.840381,15.1,79.5
Decision Tree,0.840518,0.839934,0.874059,0.849751,0.835215,0.829379,15.1,73.4
Gradient Boosting Tree,0.847637,0.847757,0.858474,0.871067,0.846137,0.818763,17.0,76.8
Support Vector Machine,0.840921,0.841445,0.807530,0.904023,0.847338,0.778947,23.1,81.4
Factorization machines,0.804717,0.799335,0.918496,0.778136,0.787202,0.860399,9.8,60.4


# מסקנות סופיות

<div style="direction:rtl">
מן הטבלה הסופית ניתן לראות כי כלל המודלים מראים תוצאות דומות בקירוב גדול מאוד, בכלל הפרמטרים. למרות זאת, נראה כי מודל random forest
הוא המודל המוביל ברוב המדדים ולכן הוא המודל המוביל לשימוש בו לצורך חיזוי אישור או דחיית אשראי של אדם.

הדיוק של 86.9% מדגים את יכולתו לחזות את תוצאת קבלת האשראי, בנוסף לזאת תוצאת הF-1 של 86.9% מצביעה על איזון טוב בין precision וrecall, דבר אשר הכרחי לטיפול אפקטיבי בהם המקרה נדחה וגם במקרים בהם אושר.

בנוסף לזאת המודל מצליח להראות הישגים מרשימים באזור מתחת לעקומה(AUC) של 86.8%, דבר המדגיש את יכולתו להתמודד גם עם תנאי סף שונים במידה וינתנו לו , ויתמודד עם רמות שונות של ודאות בסיווג.

על ידי מתן עדיפות לaccuracy ,ציון מאוזן בF-1, וביצועים מרשימים באזור מתחת לעקומה(auc), מודל הRandom forest מדגים את הפוטניצאל לסווג ולזהות את הבקשות שיאושרו ידחו בהמשך עפ"י פרמטרים שיוזנו למערכת.
</div>

### נציג את התוצאות בצורה ויזואלית נוחה יותר

In [0]:
models_df.plot.bar(x=["acc","f1","recall","precision","auc","specifity"])

### כעת נראה איזה משקל יש לכל מאפיין להחלטה האם הלקוח מאושר או לא לקבל הרשאת אשראי
### המשקל נלקח מהמודל אותו בחרנו כמתאים ביותר - Random Forest

In [0]:
# save attributes as a DF
attr_df = pd.DataFrame(results_rf.schema["features"].metadata["ml_attr"]["attrs"]["numeric"]+results_rf.schema["features"].metadata["ml_attr"]["attrs"]["binary"])

# save importance in a list
importances = fit_model_rf.stages[7].featureImportances
attr_importances = []
for i in range(len(importances)):
    attr_importances.append(importances[i])

In [0]:
# convert importance into a DF
importance_df = pd.DataFrame(attr_importances)
importance_df_final = importance_df.rename(columns={0:"importance"})

#rename attribute's column name
attr_df_bk = pd.DataFrame(attr_df['name'])

#concat both importance and attribute's DF, also sort in ascending way
importance_plot = pd.concat([importance_df_final,attr_df_bk],axis=1)
importance_plot = importance_plot.sort_values('importance',ascending=False)

In [0]:
importance_plot.plot.bar(x='name',y='importance',title = 'features importance')

<div style="direction:rtl">
ניתן לראות קוראלציה חזקה מאוד בין אם בעבר האדם התקשה בביצוע תשלומים אליהם התחייב אל אי קבלת אישור אשראי ולכן זה הוא המאפיין הראשון והמכריע על קבלת אשראי, לאחר מכן מאפיין של סטטוס העבודה של האדם וזמן ההעסקה שלו , ולבסוף המאפיין של ציון האשראי של אותו אדם המבקש אישור אשראי.

לפיכך, המלצתנו לחברה אשר נותנת אישורי אשראי להתמקד במאפיינים אלו של הלקוח, וכאשר אחד ממאפיינים אלו (או כולם) אינם מתקיימים יש לשקול דחיית אישור אשראי לאותו לקוח כדי למנוע סיכון עתידי
</div>